In [2]:
import sqlite3
import pandas as pd

In [3]:
conn =  sqlite3.connect('data/employees.db')
cur = conn.cursor()

In [51]:
cur.execute('''
CREATE TABLE IF NOT EXISTS EmployeeSalary(
            EmployeeID INTEGER PRIMARY KEY ,
            JobTitle TEXT,
            Salary INTEGER
            )       
''')

In [52]:
cur.execute(''' 
INSERT INTO EmployeeSalary(EmployeeID,JobTitle,Salary)
            VALUES(1001,'Salesman',45000),
                  (1002,'Receptionist',36000),
                 (1003,'Salesman',63000),
                 (1004,'Accountant',47000),
                 (1005,'HR',50000),
                 (1006,'Regional Manager',65000),
                 (1007,'Supplier Relations',41000),
                 (1008,'Salesman',48000),
                 (1009,'Accountant',48000),
                 (1010,NULL,47000),
                 (NULL,'Salesman',43000)
'''            )

In [53]:
q = '''
SELECT * FROM EmployeeSalary
'''
pd.read_sql(q,conn)

,EmployeeID,JobTitle,Salary
0,1001,Salesman,45000
1,1002,Receptionist,36000
2,1003,Salesman,63000
3,1004,Accountant,47000
4,1005,HR,50000
5,1006,Regional Manager,65000
6,1007,Supplier Relations,41000
7,1008,Salesman,48000
8,1009,Accountant,48000
9,1010,None,47000


In [4]:
conn.commit()

In [5]:
#test using a second connection whether data has imported
con2 = sqlite3.connect('data/employees.db')
q = '''
SELECT * FROM EmployeeSalary
'''
pd.read_sql(q,conn)

,EmployeeID,JobTitle,Salary
0,1001,Salesman,45000
1,1002,Receptionist,36000
2,1003,Salesman,63000
3,1004,Accountant,47000
4,1005,HR,50000
5,1006,Regional Manager,65000
6,1007,Supplier Relations,41000
7,1008,Salesman,48000
8,1009,Accountant,48000
9,1010,None,47000


### 1a.Subquery in select

In [56]:
q = '''
SELECT EmployeeID, Salary,(select AVG(Salary) from employeesalary) AS allAvgSalary
FROM employeesalary
'''
pd.read_sql(q,conn)

,EmployeeID,Salary,allAvgSalary
0,1001,45000,48454.545455
1,1002,36000,48454.545455
2,1003,63000,48454.545455
3,1004,47000,48454.545455
4,1005,50000,48454.545455
5,1006,65000,48454.545455
6,1007,41000,48454.545455
7,1008,48000,48454.545455
8,1009,48000,48454.545455
9,1010,47000,48454.545455


In [57]:
#alter where had put salary as 48000 instead of 42000
cur.execute('''
Update employeesalary set salary= 42000 where employeeId=1009
''')

In [58]:
conn.commit

<function Connection.commit>

### b.How to do it with Partition by

In [59]:
q = '''
SELECT EmployeeID,Salary,AVG(salary) Over() AS AllAvgSalary
FROM employeesalary
'''
pd.read_sql(q,conn)

,EmployeeID,Salary,AllAvgSalary
0,1001,45000,47909.090909
1,1002,36000,47909.090909
2,1003,63000,47909.090909
3,1004,47000,47909.090909
4,1005,50000,47909.090909
5,1006,65000,47909.090909
6,1007,41000,47909.090909
7,1008,48000,47909.090909
8,1009,42000,47909.090909
9,1010,47000,47909.090909


### c.Why group by wont work

In [60]:
q = '''
SELECT EmployeeID,Salary,AVG(salary)  AS AllAvgSalary
FROM employeesalary
GROUP BY EmployeeID,Salary
ORDER BY 1,2 --didnt get avg we are looking for 
'''
pd.read_sql(q,conn)

,EmployeeID,Salary,AllAvgSalary
0,1001,45000,45000.0
1,1002,36000,36000.0
2,1003,63000,63000.0
3,1004,47000,47000.0
4,1005,50000,50000.0
5,1006,65000,65000.0
6,1007,41000,41000.0
7,1008,48000,48000.0
8,1009,42000,42000.0
9,1010,47000,47000.0


### 2.Subquery in From


In [61]:
q = ''' 
SELECT 
    a.EmployeeID,allAvgSalary 
FROM
    (SELECT EmployeeID, Salary,(select AVG(Salary) from employeesalary) AS allAvgSalary
    FROM employeesalary) a
'''
pd.read_sql(q,conn)

,EmployeeID,allAvgSalary
0,1001,47909.090909
1,1002,47909.090909
2,1003,47909.090909
3,1004,47909.090909
4,1005,47909.090909
5,1006,47909.090909
6,1007,47909.090909
7,1008,47909.090909
8,1009,47909.090909
9,1010,47909.090909


### 3.Subquery in WHERE

In [62]:
q = ''' 
SELECT 
    EmployeeID,JobTitle,Salary
FROM employeesalary
'''
pd.read_sql(q,conn)

,EmployeeID,JobTitle,Salary
0,1001,Salesman,45000
1,1002,Receptionist,36000
2,1003,Salesman,63000
3,1004,Accountant,47000
4,1005,HR,50000
5,1006,Regional Manager,65000
6,1007,Supplier Relations,41000
7,1008,Salesman,48000
8,1009,Accountant,42000
9,1010,None,47000


### get employees over the age of 30

In [63]:
q = ''' 
SELECT 
    EmployeeID,JobTitle,Salary
FROM employeesalary
WHERE EmployeeID IN (
    SELECT EmployeeID from EmployeeDemographics
    WHERE age>30
)
'''
pd.read_sql(q,conn)

In [64]:
# cur.execute('drop table employeesalary')
# conn.commit()